# LangGraph Example

## Initialize the model and tools

In [1]:
!export LANGSMITH_TRACING=true
!export LANGSMITH_API_KEY=...

In [2]:
from typing import Annotated, Literal, TypedDict

from langchain_core.messages import HumanMessage
from langchain_ollama import ChatOllama
from langchain_core.tools import tool
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, START, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode

# The tools for the agent to use
@tool
def search(query: str):
    """Call to surf the web."""
    # This is a placeholder, but don't tell the LLM that...
    if "sf" in query.lower() or "san francisco" in query.lower():
        return "It's 60 degrees and foggy."
    return "It's 90 degrees and sunny."

tools =[search]

tool_node = ToolNode(tools)

model = ChatOllama(model="llama3.1", temperature=0).bind_tools(tools)

## Initialize graph with state

In [3]:
# The function that determines whether to continue or not
def should_continue(state: MessagesState) -> Literal["tools", END]:
    messages = state['messages']
    last_message = messages[-1]
    # If the LLM makes a tool call, then we route to the "tools" node
    if last_message.tool_calls:
        return "tools"
    # Otherwise, we stop(reply to the user)
    return END

# The function that calls the model
def call_model(state: MessagesState):
    messages = state['messages']
    response = model.invoke(messages)
    # We return a list, bc this will get added to the existing list
    return {"messages": [response]}

# A new graph
workflow = StateGraph(MessagesState)

## Define graph nodes

In [4]:

# The 2 nodes we will cycle btw
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

## Define entry point and graph edges

In [5]:
# Set the entrypoint as 'agent'
# This means that this node is the 1st one called
workflow.add_edge(START, "agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # 1st, we define the start node. We use 'agent'.
    # These are the edges taken afger the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
)

# We now add a normal edge from 'tools' to `agent`.
# After 'tools' is called, 'agent' node is called next.
workflow.add_edge("tools", 'agent')

## Compile the graph

In [6]:
# Initialize memory to persist state between graph runs
checkpointer = MemorySaver()

# Finally, this compiles it into LangChain Runnable,
# Note We are passing the memory when compiling the graph
app = workflow.compile(checkpointer=checkpointer)

## Execute the graph

In [8]:
# Use the Runnable
final_state = app.invoke(
    {"messages": [HumanMessage(content="What is the weather in sf")]},
    config={"configurable": {"thread_id": 42}}
)
final_state["messages"][-1].content

'Based on the tool call response, I can provide a more detailed answer to your original question:\n\nThe current weather in San Francisco (SF) is 60 degrees Fahrenheit with foggy conditions.'

In [9]:
final_state = app.invoke(
    {"messages": [HumanMessage(content="what about ny")]},
    config={"configurable": {"thread_id": 42}}
)
final_state["messages"][-1].content

'Based on the tool call response, I can provide a more detailed answer to your original question:\n\nThe current weather in New York (NY) is 90 degrees Fahrenheit with clear skies and sunny conditions.'